In [6]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
root_dir=os.getcwd()
src_path_train = os.path.join(root_dir,"../data/chest_xray/train")
src_path_test = os.path.join(root_dir,"../data/chest_xray/test")
src_path_val = os.path.join(root_dir,"../data/chest_xray/val")
print(os.path.exists(src_path_test))
print(os.path.exists(src_path_train))

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)
val_datagen = ImageDataGenerator(rescale=1 / 255.0)

True
True


In [7]:

train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    directory=src_path_val,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [8]:
def prepare_model_corrected():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    # --- Corrected Layer ---
    model.add(Dense(1, activation='sigmoid')) # 1 neuron for binary output
    
    model.compile(loss="binary_crossentropy", # Correct loss for this setup
                  optimizer="adam",
                  metrics=['accuracy'])
    return model

In [9]:
model=prepare_model_corrected()
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=5)


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 270ms/step - accuracy: 0.7092 - loss: 0.7931 - val_accuracy: 0.5000 - val_loss: 0.6957
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 275ms/step - accuracy: 0.7485 - loss: 0.6527 - val_accuracy: 0.5000 - val_loss: 0.7030
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 319ms/step - accuracy: 0.7531 - loss: 0.6267 - val_accuracy: 0.5000 - val_loss: 0.7131
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 47s 289ms/step - accuracy: 0.7424 - loss: 0.6120 - val_accuracy: 0.5000 - val_loss: 0.7253
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 274ms/step - accuracy: 0.7368 - loss: 0.6019 - val_accuracy: 0.5000 - val_loss: 0.7382


In [ ]:
import pickle


with open('training_history.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)